<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/guides/getting-started/v2/tutorial_pt6_v2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# RAG

The Chat endpoint provides comprehensive support for various text generation use cases, including retrieval-augmented generation (RAG). 

While LLMs are good at maintaining the context of the conversation and generating responses, they can be prone to hallucinate and include factually incorrect or incomplete information in their responses.

RAG enables a model to access and utilize supplementary information from external documents, thereby improving the accuracy of its responses.

When using RAG with the Chat endpoint, these responses are backed by fine-grained citations linking to the source documents. This makes the responses easily verifiable.

In this tutorial, you'll learn about:
- Basic RAG
- Search query generation
- Retrieval with Embed
- Reranking with Rerank
- Response and citation generation

You'll learn these by building an onboarding assistant for new hires.

## Setup

To get started, first we need to install the `cohere` library and create a Cohere client.

In [9]:

# pip install cohere

import cohere
import numpy as np
import json
from typing import List

co = cohere.ClientV2(api_key="COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

## Basic RAG


To see how RAG works, let's define the documents that the application has access to. We'll use a short list of documents consisting of internal FAQs about the fictitious company Co1t (in production, these documents are massive).

In this example, each document is a `data` object with one field, `text`. But we can define any number of fields we want, depending on the nature of the documents. For example, emails could contain `title` and `text` fields.

In [3]:
documents = [
  {
    "data": {
      "text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."
    }
  },
  {
    "data": {
      "text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."
    }
  },
  {
    "data": {
      "text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."
    }
  }
]

To call the Chat API with RAG, pass the following parameters at a minimum. This tells the model to run in RAG-mode and use these documents in its response.

- `model` for the model ID
- `messages` for the user's query.
- `documents` for defining the documents.

Let's create a query asking about the company's support for personal well-being, which is not going to be available to the model based on the data its trained on. It will need to use external documents.

RAG introduces additional objects in the Chat response. One of them is `citations`, which contains details about:
- specific text spans from the retrieved documents on which the response is grounded.
- the documents referenced in the citations.

In [37]:
# Add the user query
query = "Are there health benefits?"

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=[{'role': 'user', 'content': query}],
                   documents=documents)

# Display the response
print(response.message.content[0].text)

# Display the citations and source documents
if response.message.citations:
    print("\nCITATIONS:")
    for citation in response.message.citations:
        print(citation, "\n")

## Search query generation

The previous example showed how to get started with RAG, and in particular, the augmented generation portion of RAG. But as its name implies, RAG consists of other steps, such as retrieval. 

In a basic RAG application, the steps involved are:

- Transforming the user message into search queries
- Retrieving relevant documents for a given search query
- Generating the response and citations

Let's now look at the first step—search query generation. The chatbot needs to generate an optimal set of search queries to use for retrieval. 

There are different possible approaches to this. In this example, we'll take a [tool use](v2/docs/tool-use) approach.

Here, we build a tool that takes a user query and returns a list of relevant document snippets for that query. The tool can generate zero, one or multiple search queries depending on the user query.

We'll not do a detailed walkthrough of tool use here as we'll do that in the next tutorial, which is a dedicated tutorial on tool use.

We recommend using the v1 API for this functionality in order to leverage the `force_single_step` feature, as follows:

In [26]:
co_v1 = cohere.Client(api_key="COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

def generate_search_queries(message: str) -> List[str]:
    
    # Define the query generation tool
    query_gen_tool = [
    {
        "name": "internet_search",
        "description": "Returns a list of relevant document snippets for a textual query retrieved from the internet",
        "parameter_definitions": {
            "queries": {
                "description": "a list of queries to search the internet with.",
                "type": "List[str]",
                "required": True
            }
        }
    }]

    # Define a preamble to optimize search query generation
    instructions = "Write a search query that will find helpful information for answering the user's question accurately. If you need more than one search query, write a list of search queries. If you decide that a search is very unlikely to find information that would be useful in constructing a response to the user, you should instead directly answer."

    # Generate search queries (if any)
    search_queries = []

    response = co_v1.chat(
        preamble=instructions,
        model="command-r-08-2024",
        message=message,
        force_single_step=True,
        tools=query_gen_tool
    )

    if response.tool_calls:
        search_queries = response.tool_calls[0].parameters["queries"]

    return search_queries

In the example above, the tool breaks down the user message into two separate queries.

In [27]:
query = "How to stay connected with the company, and do you organize team events?"
queries_for_search = generate_search_queries(query)
print(queries_for_search)

['how to stay connected with the company', 'does the company organize team events']


And in the example below, the tool decides that one query is sufficient.

In [28]:
query = "How flexible are the working hours"
queries_for_search = generate_search_queries(query)
print(queries_for_search)

['how flexible are the working hours at the company']


And in the example below, the tool decides that no retrieval is needed to answer the query.

In [29]:
query = "What is 2 + 2"
queries_for_search = generate_search_queries(query)
print(queries_for_search)

[]


## Retrieval with Embed

Given the search query, we need a way to retrieve the most relevant documents from a large collection of documents.

This is where we can leverage text embeddings through the Embed endpoint. It enables semantic search, which lets us to compare the semantic meaning of the documents and the query. It solves the problem faced by the more traditional approach of lexical search, which is great at finding keyword matches, but struggles at capturing the context or meaning of a piece of text.

The Embed endpoint takes in texts as input and returns embeddings as output.

First, we need to embed the documents to search from. We call the Embed endpoint using `co.embed()` and pass the following arguments:

- `model`: Here we choose `embed-english-v3.0`, which generates embeddings of size 1024
- `input_type`: We choose `search_document` to ensure the model treats these as the documents (instead of the query) for search
- `texts`: The list of texts (the FAQs)

In [30]:
# Define the documents
faqs_long = [
    {"text": "Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged."},
    {"text": "Finding Coffee Spots: For your caffeine fix, head to the break room's coffee machine or cross the street to the café for artisan coffee."},
    {"text": "Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!"},
    {"text": "Working Hours Flexibility: We prioritize work-life balance. While our core hours are 9 AM to 5 PM, we offer flexibility to adjust as needed."},
    {"text": "Side Projects Policy: We encourage you to pursue your passions. Just be mindful of any potential conflicts of interest with our business."},
    {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
    {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."},
    {"text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."},
    {"text": "Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year."},
    {"text": "Proposing New Ideas: Innovation is welcomed! Share your brilliant ideas at our weekly team meetings or directly with your team lead."},
]

# Embed the documents
doc_emb = co.embed(
            model="embed-english-v3.0",
            input_type="search_document",
            texts=[doc['text'] for doc in faqs_long],
            embedding_types=["float"]).embeddings.float

Next, we add a query, which asks about how to get to know the team.

We choose `search_query` as the `input_type` to ensure the model treats this as the query (instead of the documents) for search.

In [31]:
# Add the user query
query = "How to get to know my teammates"

# Generate the search query
# Note: For simplicity, we are assuming only one query generated. For actual implementations, you will need to perform search for each query.
queries_for_search = generate_search_queries(query)[0]
print("Search query: ", queries_for_search)

# Embed the search query
query_emb = co.embed(
    model="embed-english-v3.0",
    input_type="search_query",
    texts=[queries_for_search],
    embedding_types=["float"]).embeddings.float

Search query:  how to get to know teammates


Now, we want to search for the most relevant documents to the query. For this, we make use of the `numpy` library to compute the similarity between each query-document pair using the dot product approach.

Each query-document pair returns a score, which represents how similar the pair are. We then sort these scores in descending order and select the top most similar pairs, which we choose 5 (this is an arbitrary choice, you can choose any number).

Here, we show the most relevant documents with their similarity scores.

In [32]:
# Compute dot product similarity and display results
n = 5
scores = np.dot(query_emb, np.transpose(doc_emb))[0]
max_idx = np.argsort(-scores)[:n]

retrieved_documents = [faqs_long[item] for item in max_idx]

for rank, idx in enumerate(max_idx):
    print(f"Rank: {rank+1}")
    print(f"Score: {scores[idx]}")
    print(f"Document: {retrieved_documents[rank]}\n")


Rank: 1
Score: 0.34212792245283796
Document: {'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}

Rank: 2
Score: 0.2883222063024371
Document: {'text': 'Proposing New Ideas: Innovation is welcomed! Share your brilliant ideas at our weekly team meetings or directly with your team lead.'}

Rank: 3
Score: 0.278128283997032
Document: {'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}

Rank: 4
Score: 0.19474858706643985
Document: {'text': "Finding Coffee Spots: For your caffeine fix, head to the break room's coffee machine or cross the street to the café for artisan coffee."}

Rank: 5
Score: 0.13713692506528824
Document: {'text': 'Side Projects Policy: We encourage you to pursue your passions. Just be mindful of any potential conflicts of interest with our business.'}



Reranking can boost the results from semantic or lexical search further. The Rerank endpoint takes a list of search results and reranks them according to the most relevant documents to a query. This requires just a single line of code to implement.

We call the endpoint using `co.rerank()` and pass the following arguments:

- `query`: The user query
- `documents`: The list of documents we get from the semantic search results
- `top_n`: The top reranked documents to select
- `model`: We choose Rerank English 3

Looking at the results, we see that the given a query about getting to know the team, the document that talks about joining Slack channels is now ranked higher (1st) compared to earlier (3rd).

Here we select `top_n` to be 2, which will be the documents we will pass next for response generation.

In [33]:
# Rerank the documents
results = co.rerank(query=queries_for_search,
                    documents=retrieved_documents,
                    top_n=2,
                    model='rerank-english-v3.0')

# Display the reranking results
for idx, result in enumerate(results.results):
    print(f"Rank: {idx+1}") 
    print(f"Score: {result.relevance_score}")
    print(f"Document: {retrieved_documents[result.index]}\n")
    
reranked_documents = [retrieved_documents[result.index] for result in results.results]

Rank: 1
Score: 0.0020507434
Document: {'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}

Rank: 2
Score: 0.0014158706
Document: {'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}



Finally we reach the step that we saw in the earlier `Basic RAG` section.

To call the Chat API with RAG, we pass the following parameters. This tells the model to run in RAG-mode and use these documents in its response.

- `model` for the model ID
- `messages` for the user's query.
- `documents` for defining the documents.

The response is then generated based on the the query and the documents retrieved.

RAG introduces additional objects in the Chat response. One of them is `citations`, which contains details about:
- specific text spans from the retrieved documents on which the response is grounded.
- the documents referenced in the citations.

In [24]:
# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=[{'role': 'user', 'content': query}],
                   documents=reranked_documents)

# Display the response
print(response.message.content[0].text)

# Display the citations and source documents
if response.message.citations:
    print("\nCITATIONS:")
    for citation in response.message.citations:
        print(citation, "\n")

You can get to know your teammates by joining your company's Slack channels and taking part in team-building activities. You will receive an invite to join relevant Slack channels via email. Your company also organises monthly outings and weekly game nights to foster team spirit.

CITATIONS:
start=38 end=75 text="joining your company's Slack channels" sources=[Source_Document(id='doc:0:0', document={'id': 'doc:0:0', 'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}, type='document')] 

start=95 end=120 text='team-building activities.' sources=[Source_Document(id='doc:0:1', document={'id': 'doc:0:1', 'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}, type='document')] 

start=130 end=190 text='receive an invite to join relevant Slack channels via email.' sources=[Source_Document(id='doc:0:0', documen